In [37]:
import jupyter_black

In [38]:
jupyter_black.load()

In [39]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Parameters

In [40]:
jan_data_uri, feb_data_uri = [
    "./data/yellow_tripdata_2022-01.parquet",
    "./data/yellow_tripdata_2022-02.parquet",
]

# Load the data

In [75]:
jan_df = pd.read_parquet(jan_data_uri)
feb_df = pd.read_parquet(feb_data_uri)

# Explore the data

In [42]:
jan_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [43]:
feb_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [44]:
len(jan_df.columns), len(feb_df.columns)

(19, 19)

In [45]:
jan_df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [46]:
feb_df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [47]:
(jan_df.dtypes == feb_df.dtypes).all()

True

# Q1
How many columns in the January data?

In [48]:
len(jan_df.columns)

19

# Q2
Standard deviation of duration for January data.
According to the homework spec, the column should be called 'duration' and this should be the duration in minutes.

In [49]:
jan_df["duration"] = (
    jan_df["tpep_dropoff_datetime"] - jan_df["tpep_pickup_datetime"]
).dt.total_seconds() / 60

In [50]:
jan_df["duration"].std()

46.44530513776499

# Q3 
Removing duration outliers

In [51]:
jan_no_duration_outliers_df = jan_df.query("1 <= duration <= 60").copy()

Percentage of rows remaining after removing the duration outliers:

In [52]:
(len(jan_no_duration_outliers_df) / len(jan_df)) * 100

98.27547930522405

# Q4

In [58]:
jan_no_duration_outliers_df[["PULocationID", "DOLocationID"]].head().to_dict(
    orient="records"
)

[{'PULocationID': 142, 'DOLocationID': 236},
 {'PULocationID': 236, 'DOLocationID': 42},
 {'PULocationID': 166, 'DOLocationID': 166},
 {'PULocationID': 114, 'DOLocationID': 68},
 {'PULocationID': 68, 'DOLocationID': 163}]

In [59]:
dv = DictVectorizer()

In [60]:
vectorized_locations_jan = dv.fit_transform(
    jan_no_duration_outliers_df[["PULocationID", "DOLocationID"]]
    .astype("str")
    .to_dict(orient="records"),
)

In [61]:
vectorized_locations_jan.shape

(2421440, 515)

# Q5
Train a linear model to predict the trip duration.

The problem specification expects us to not scale the input to standardise or normalise it,
so we retain the input features as they are.

In [62]:
lr = LinearRegression()

In [63]:
lr.fit(
    X=vectorized_locations_jan,
    y=jan_no_duration_outliers_df["duration"],
)

LinearRegression()

In [64]:
y_pred = lr.predict(vectorized_locations_jan)

In [65]:
rmse = mean_squared_error(
    y_pred, jan_no_duration_outliers_df["duration"], squared=False
)

In [66]:
rmse

6.9861908469917315

# Q6

In [76]:
feb_df["duration"] = (
    feb_df["tpep_dropoff_datetime"] - feb_df["tpep_pickup_datetime"]
).dt.total_seconds() / 60

In [77]:
feb_df["duration"].std()

47.26394076781901

In [78]:
feb_no_duration_outliers_df = feb_df.query("1 <= duration <= 60").copy()

Percentage of rows remaining after removing outliers in duration:

In [79]:
(len(feb_no_duration_outliers_df) / len(feb_df)) * 100

97.9444397269143

In [80]:
vectorized_locations_feb = dv.transform(
    feb_no_duration_outliers_df[["PULocationID", "DOLocationID"]]
    .astype("str")
    .to_dict(orient="records")
)

In [88]:
y_pred_feb = lr.predict(vectorized_locations_feb)

In [89]:
rmse_feb = mean_squared_error(
    y_pred_feb, feb_no_duration_outliers_df["duration"], squared=False
)

In [90]:
rmse_feb

7.786406838555644

## What is the RMSE on the feb data without removing duration outliers?
This is actually not required by the problem specification, but I was curious to know.

In [ ]:
vectorized_locations_feb_with_outliers = dv.transform(
    feb_df[["PULocationID", "DOLocationID"]].astype("str").to_dict(orient="records")
)

In [85]:
y_pred_feb_with_outliers = lr.predict(vectorized_locations_feb_with_outliers)

In [86]:
rmse_feb_with_outliers = mean_squared_error(
    y_pred_feb_with_outliers, feb_df["duration"], squared=False
)

In [87]:
rmse_feb_with_outliers

46.877270673288834